## Part II: Physical Database Design and SQL Retrieval


###  1. Final Schema Decisions

### ER & Schema Decisions
- Central entity: INCIDENT
- Offenses, arrestees, victims, and properties are linked to INCIDENT
- Lookup tables defined for codes and categories
- Many-to-many relationships handled using junction tables (BIAS_MOTIVATION, VICTIM_OFFENSE, PROPERTY_DESC)
- Standardized age and sex codes across tables
- Composite primary keys applied where needed

### Disagreements & Resolutions
- **ARRESTEE.age_num**: Initially VARCHAR, resolved to INT for consistency with VICTIM
- **VICTIM.age_code_range_high** vs ARRESTEE.age_range_high_num: Unified to `age_range_high_num`
- **Duplicate OFFENSE table definitions**: Resolved by keeping the extended version with foreign keys
- **Sex/resident codes consistency**: Agreed on CHAR(1) standard

### Unresolved Items
- None

In [ ]:
!unzip -l "/home/ubuntu/Assignment_2/MD_2024_NIBRS_DB.zip"
## here we listed the contents of the zip file without extracting it to confirm whats inside and check that all the expected data is there before unzipping

Archive:  /home/ubuntu/Assignment_2/MD_2024_NIBRS_DB.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
        0  2025-10-14 11:23   datafiles/
  3759376  2025-10-06 18:07   datafiles/NIBRS_ARRESTEE.csv
      105  2025-10-06 18:07   datafiles/NIBRS_ARREST_TYPE.csv
     1623  2025-10-06 18:07   datafiles/NIBRS_BIAS_LIST.csv
  5424581  2025-10-06 18:07   datafiles/NIBRS_BIAS_MOTIVATION.csv
      141  2025-10-06 18:07   datafiles/NIBRS_ETHNICITY.csv
 25342579  2025-10-06 18:07   datafiles/NIBRS_incident.csv
 10853371  2025-10-06 18:07   datafiles/NIBRS_OFFENSE.csv
     5428  2025-10-06 18:07   datafiles/NIBRS_OFFENSE_TYPE.csv
 15681825  2025-10-06 18:07   datafiles/NIBRS_VICTIM.csv
  8416881  2025-10-06 18:07   datafiles/NIBRS_VICTIM_OFFENSE.csv
      214  2025-10-06 18:07   datafiles/NIBRS_VICTIM_TYPE.csv
    13285  2025-10-07 17:52   datafiles/README.md
        0  2025-10-14 11:23   datafiles/Sponsor Request/
  7057548  2025-10-07 18:21   datafiles/Sponsor Reques

In [ ]:
!unzip -o "/home/ubuntu/Assignment_2/MD_2024_NIBRS_DB.zip" -d "/home/ubuntu/Assignment_2/NIBRS_files"
## This extracts the ZIP file into a specific folder named NIBRS_files. The -o flag allows overwriting any existing files in that directory. This prepares all the data files for later use when creating the database.

Archive:  /home/ubuntu/Assignment_2/MD_2024_NIBRS_DB.zip
   creating: /home/ubuntu/Assignment_2/NIBRS_files/datafiles/
  inflating: /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ARRESTEE.csv  
  inflating: /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ARREST_TYPE.csv  
  inflating: /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_BIAS_LIST.csv  
  inflating: /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_BIAS_MOTIVATION.csv  
  inflating: /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ETHNICITY.csv  
  inflating: /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_incident.csv  
  inflating: /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_OFFENSE.csv  
  inflating: /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_OFFENSE_TYPE.csv  
  inflating: /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_VICTIM.csv  
  inflating: /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_VICTIM_OFFENSE.csv  
  inflating: /home/ubuntu/Assignment_2/NIB

In [ ]:
# Main NIBRS tables,This previews the first 5 lines of a few main data files. The goal is to inspect column names and sample data so you can understand the structure of each dataset before creating tables. This step supports the ER diagram and schema design.
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ARRESTEE.csv"
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ARREST_TYPE.csv"
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_BIAS_LIST.csv"
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_BIAS_MOTIVATION.csv"
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ETHNICITY.csv"
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_incident.csv"
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_OFFENSE.csv"
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_OFFENSE_TYPE.csv"
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_VICTIM.csv"
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_VICTIM_OFFENSE.csv"
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_VICTIM_TYPE.csv"


data_year,arrestee_id,incident_id,arrestee_seq_num,arrest_date,arrest_type_id,multiple_indicator,offense_code,age_id,age_num,sex_code,race_id,ethnicity_id,resident_code,under_18_disposition_code,clearance_ind,age_range_low_num,age_range_high_num
2024,59202696,194040775,1,2024-04-22,2,N,90Z,36,33,M,10,20,R,,,33,
2024,59202697,194040776,1,2024-01-04,1,N,35A,36,33,M,10,20,R,,,33,
2024,56110705,184397471,1,2024-01-04,1,N,23H,29,26,F,10,20,R,,,26,
2024,56733190,186428152,1,2024-01-25,3,N,220,45,42,F,10,20,R,,,42,
arrest_type_id,arrest_type_code,arrest_type_name
1,O,On View
2,S,Summoned / Cited
3,T,Taken INTO Custody
bias_id,bias_code,bias_category,bias_desc
11,11,Race/Ethnicity/Ancestry,Anti-White
12,12,Race/Ethnicity/Ancestry,Anti-Black or African American
13,13,Race/Ethnicity/Ancestry,Anti-American Indian or Alaska Native
14,14,Race/Ethnicity/Ancestry,Anti-Asian
data_year,bias_id,offense_id
2024,88,230427687
2024,88,230427688
2024,88,219312155
2024,88,218839132
ethnicity_id,ethnicity_code

In [ ]:
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROPERTY.csv"
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROPERTY_DESC.csv"
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROP_DESC_TYPE.csv"
!head -n 5 "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROP_LOSS_TYPE.csv"
## This previews the four property-related files mentioned in the sponsor request section of the assignment. By checking the headers, you can understand what each file contains and how they might integrate with the existing NIBRS tables.

data_year,property_id,incident_id,prop_loss_id,stolen_count,recovered_count
2024,187154350,194040775,7,,
2024,187154351,194040776,6,,
2024,178922509,184397471,1,,
2024,178560128,183991844,1,,
data_year,property_id,prop_desc_id,property_value,date_recovered,nibrs_prop_desc_id
2024,187154350,20,2700,,
2024,187154351,10,,,
2024,178560129,63,30,,
2024,178922510,10,,,
prop_desc_id,prop_desc_name,prop_desc_code
1,Aircraft,01
2,Alcohol,02
3,Automobile,03
4,Bicycles,04
prop_loss_id,prop_loss_name,prop_loss_desc
1,None,None
2,Burned,Burned (includes damage caused in fighting the fire)
3,Counterfeited/Forged,Counterfeited/Forged
4,Destroyed/Damaged/Vandalized,Destroyed/Damaged/Vandalized


In [ ]:
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROPERTY.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROPERTY_DESC.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROP_DESC_TYPE.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROP_LOSS_TYPE.csv"
## The wc -l command counts the number of lines in each file. This helped us verify the total number of rows before importing them into the database.

242273 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROPERTY.csv
292456 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROPERTY_DESC.csv
69 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROP_DESC_TYPE.csv
9 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROP_LOSS_TYPE.csv


### 2. Create database objects

In [ ]:
%load_ext sql

In [ ]:
%sql postgresql://student@/postgres

Connecting to 'postgresql://student@/postgres'

In [ ]:
!dropdb  -U stu

dropdb: error: missing required argument database name
dropdb: hint: Try "dropdb --help" for more information.


In [ ]:
%%sql
DROP DATABASE IF EXISTS nibrs_db;
CREATE DATABASE nibrs_db;

Running query in 'postgresql://student@/postgres'

++
||
++
++

In [ ]:
%sql postgresql://student@/nibrs_db

Connecting and switching to connection 'postgresql://student@/nibrs_db'

- This section sets up the core database foundation required for analysis. Once the tables are created, data can be loaded into them, and queries can be run to answer questions like the frequency of offenses, victim demographics, and incident patterns.

- Each table represents a key component of the crime reporting system (for example, INCIDENT for crime events, VICTIM for those affected,and OFFENSE for specific crimes).

- Primary keys (like incident_id, victim_id, offense_id, etc.) ensure that every record is uniquely identifiable.

- Foreign keys connect related tables, allowing relationships such as:

- A victim belonging to an incident (incident_id), An offense linked to its offense type (offense_code), A relationship connecting a victim and an offender.

In [ ]:
%%sql
-- Drop the ARREST_TYPE table if it already exists to avoid duplication
DROP TABLE IF EXISTS ARREST_TYPE CASCADE;

-- Create the ARREST_TYPE table
CREATE TABLE ARREST_TYPE (
    arrest_type_id INT NOT NULL,         -- Unique identifier for each arrest type (Primary Key)
    arrest_type_code VARCHAR(10),        -- Short code or abbreviation for the arrest type
    arrest_type_name VARCHAR(50),        -- Descriptive name of the arrest type
    PRIMARY KEY (arrest_type_id)         -- Make sure each arrest_type_id is unique within the table
);


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Add comments for table and columns
COMMENT ON TABLE ARREST_TYPE IS 'Table containing types of arrests in the NIBRS dataset';
COMMENT ON COLUMN ARREST_TYPE.arrest_type_id IS 'Unique identifier for each arrest type';
COMMENT ON COLUMN ARREST_TYPE.arrest_type_code IS 'Abbreviated code representing the arrest type';
COMMENT ON COLUMN ARREST_TYPE.arrest_type_name IS 'Full descriptive name of the arrest type';

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the ARRESTEE table if it already exists to avoid duplication
DROP TABLE IF EXISTS ARRESTEE CASCADE;

-- Create the ARRESTEE table
CREATE TABLE ARRESTEE (
    data_year INT,                               -- Year the arrest data was recorded
    arrestee_id BIGINT NOT NULL,                  -- Unique identifier for each arrestee (Primary Key)
    incident_id BIGINT,                          -- Identifier linking the arrestee to a specific incident record
    arrestee_seq_num INT,                        -- Sequence number in case multiple arrestees are tied to one incident
    arrest_date DATE,                            -- Date of arrest
    arrest_type_id INT,                          -- Foreign key referencing ARREST_TYPE
    multiple_indicator CHAR(1),                   -- Tells us if the arrestee was involved in multiple offenses
    offense_code VARCHAR(10),                     -- Representing the specific offense for which the arrestee was charged
    age_id INT,                                  -- Identifier for age group or category
    age_num INT,                                 -- Numeric age of the arrestee
    sex_code CHAR(1),                            -- Gender of the arrestee
    race_id INT,                                 -- Identifier referencing a race lookup table
    ethnicity_id INT,                            -- Identifier referencing an ethnicity lookup table
    resident_code CHAR(1),                       -- Tells us residency status
    under_18_disposition_code CHAR(5),           -- Describing legal disposition if the arrestee is under 18
    clearance_ind CHAR(1),                       -- Tells us if the arrest cleared the case
    age_range_low_num INT,                       -- Lower bound of age range
    age_range_high_num INT,                      -- Upper bound of age range
    PRIMARY KEY (arrestee_id),                    -- Ensure each arrestee_id is unique
    FOREIGN KEY (arrest_type_id) REFERENCES ARREST_TYPE(arrest_type_id)  -- Link to ARREST_TYPE table for arrest categories
);


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE ARRESTEE IS 'Table containing detailed information for each arrestee in NIBRS dataset';
COMMENT ON COLUMN ARRESTEE.data_year IS 'Year in which the arrest occurred';
COMMENT ON COLUMN ARRESTEE.arrestee_id IS 'Unique identifier for each arrestee';
COMMENT ON COLUMN ARRESTEE.incident_id IS 'ID linking the arrestee to a specific incident';
COMMENT ON COLUMN ARRESTEE.arrestee_seq_num IS 'Sequence number for multiple arrestees in the same incident';
COMMENT ON COLUMN ARRESTEE.arrest_date IS 'Date when the arrest occurred';
COMMENT ON COLUMN ARRESTEE.arrest_type_id IS 'Reference to the type of arrest from ARREST_TYPE table';
COMMENT ON COLUMN ARRESTEE.multiple_indicator IS 'Indicates if the arrestee was involved in multiple offenses';
COMMENT ON COLUMN ARRESTEE.offense_code IS 'Code representing the offense for which the arrestee was charged';
COMMENT ON COLUMN ARRESTEE.age_id IS 'Identifier for age group';
COMMENT ON COLUMN ARRESTEE.age_num IS 'Numeric age of the arrestee';
COMMENT ON COLUMN ARRESTEE.sex_code IS 'Gender of the arrestee';
COMMENT ON COLUMN ARRESTEE.race_id IS 'Reference to race lookup table';
COMMENT ON COLUMN ARRESTEE.ethnicity_id IS 'Reference to ethnicity lookup table';
COMMENT ON COLUMN ARRESTEE.resident_code IS 'Indicates residency status of arrestee';
COMMENT ON COLUMN ARRESTEE.under_18_disposition_code IS 'Legal disposition if arrestee is under 18';
COMMENT ON COLUMN ARRESTEE.clearance_ind IS 'Indicates if the arrest cleared the case';
COMMENT ON COLUMN ARRESTEE.age_range_low_num IS 'Lower bound of age range for grouping';
COMMENT ON COLUMN ARRESTEE.age_range_high_num IS 'Upper bound of age range for grouping';

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the BIAS_LIST table if it already exists to avoid duplication
DROP TABLE IF EXISTS BIAS_LIST CASCADE;

-- Create the BIAS_LIST table
CREATE TABLE BIAS_LIST (
    bias_id INT NOT NULL,                    -- Unique identifier for each bias type (Primary Key)
    bias_code VARCHAR(10),                   -- Short code used to represent the bias (RELG2)
    bias_category VARCHAR(50),               -- General category of bias (Race/Ethnicity)
    bias_desc VARCHAR(100),                 -- Detailed description of the bias (Anti-Black)
    PRIMARY KEY (bias_id)                    -- Ensures each bias_id is unique across the table
);


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE BIAS_LIST IS 'Table containing types of biases with codes, categories, and descriptions';
COMMENT ON COLUMN BIAS_LIST.bias_id IS 'Unique identifier for each bias type';
COMMENT ON COLUMN BIAS_LIST.bias_code IS 'Short code representing the bias type';
COMMENT ON COLUMN BIAS_LIST.bias_category IS 'General category of the bias (e.g., Race/Ethnicity, Religion)';
COMMENT ON COLUMN BIAS_LIST.bias_desc IS 'Detailed textual description of the bias';

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the OFFENSE table if it already exists to avoid duplicaton
DROP TABLE IF EXISTS OFFENSE CASCADE;

-- Create the OFFENSE table
CREATE TABLE OFFENSE (
    data_year INT,                           -- Year offense was recorded
    offense_id BIGINT PRIMARY KEY,            -- Unique identifier for each offense record
    incident_id BIGINT,                       -- Identifier linking the offense to a specific incident
    offense_code VARCHAR(10),                 -- Code representing the type of offense
    attempt_complete_flag CHAR(1),            -- Indicates the status of the offense: 'A' = Attempted, 'C' = Completed, telling us whether the crime was successfully carried out or only attempted
    location_id INT                           -- Foreign key referencing the location where the offense occurred
);

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE OFFENSE IS 'Table containing offense details for each incident in the NIBRS dataset';
COMMENT ON COLUMN OFFENSE.data_year IS 'Year in which the offense was recorded';
COMMENT ON COLUMN OFFENSE.offense_id IS 'Unique identifier for each offense record';
COMMENT ON COLUMN OFFENSE.incident_id IS 'ID linking the offense to a specific incident';
COMMENT ON COLUMN OFFENSE.offense_code IS 'Code representing the type of offense';
COMMENT ON COLUMN OFFENSE.attempt_complete_flag IS 'Indicates if the offense was attempted or completed';
COMMENT ON COLUMN OFFENSE.location_id IS 'ID of the location where the offense occurred';

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the BIAS_MOTIVATION table if it already exists to avoid duplication
DROP TABLE IF EXISTS BIAS_MOTIVATION CASCADE;

-- Create the BIAS_MOTIVATION table
CREATE TABLE BIAS_MOTIVATION (
    data_year INT,                                           -- Year the bias motivation was reported
    bias_id INT,                                             -- Foreign key referencing the bias type from BIAS_LIST
    offense_id BIGINT,                                       -- Foreign key referencing the related offense
    PRIMARY KEY (bias_id, offense_id),                       -- Primary key ensures uniqueness for each bias/offense pair
    FOREIGN KEY (bias_id) REFERENCES BIAS_LIST(bias_id),    -- Link to BIAS_LIST for bias categories
    FOREIGN KEY (offense_id) REFERENCES OFFENSE(offense_id) -- Link to OFFENSE for details of the biased offense
);


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE BIAS_MOTIVATION IS 'Table linking biases to specific offenses, indicating bias motivation';
COMMENT ON COLUMN BIAS_MOTIVATION.data_year IS 'Year in which the bias motivation was reported';
COMMENT ON COLUMN BIAS_MOTIVATION.bias_id IS 'Identifier for the type of bias, referencing BIAS_LIST';
COMMENT ON COLUMN BIAS_MOTIVATION.offense_id IS 'Identifier for the offense, referencing OFFENSE';


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the ETHNICITY table if it already exists to avoid duplication
DROP TABLE IF EXISTS ETHNICITY CASCADE;

-- Create the ETHNICITY table
CREATE TABLE ETHNICITY (
    ethnicity_id INT NOT NULL,               -- Unique identifier for each ethnicity (Primary Key)
    ethnicity_code CHAR(1),                  -- Short code for ethnicity ('H' = Hispanic, 'N' = Not Hispanic)
    ethnicity_name VARCHAR(50),              -- Full descriptive name of the ethnicity
    PRIMARY KEY (ethnicity_id)               -- Makes sure each ethnicity_id is unique in the table
);


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE ETHNICITY IS 'Table containing ethnicity categories used in the NIBRS dataset';
COMMENT ON COLUMN ETHNICITY.ethnicity_id IS 'Unique identifier for each ethnicity';
COMMENT ON COLUMN ETHNICITY.ethnicity_code IS 'Short code representing the ethnicity';
COMMENT ON COLUMN ETHNICITY.ethnicity_name IS 'Full descriptive name of the ethnicity';

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the INCIDENT table if it already exists to avoid duplication
DROP TABLE IF EXISTS INCIDENT CASCADE;

-- Create the INCIDENT table
CREATE TABLE INCIDENT (
    data_year INT,                             -- Year the incident data was recorded
    agency_id INT,                             -- Identifier for the reporting law enforcement agency
    incident_id BIGINT NOT NULL,               -- Unique identifier for each incident (Primary Key)
    nibrs_month_id BIGINT,                     -- Identifier linking to the NIBRS reporting month
    cargo_theft_flag CHAR(1),                  -- Indicates if the incident involved cargo theft
    submission_date TIMESTAMP,                 -- Date and time when the incident was submitted to the database/system
    incident_date DATE,                        -- Actual date when the incident occurred
    report_date_flag CHAR(1),                  -- Tells us whether the reported date differs from the incident date
    incident_hour INT,                         -- Hour of the day when the incident occurred
    cleared_except_id INT,                     -- ID referencing the reason for exceptional clearance
    cleared_except_date DATE,                  -- Date when case was exceptionally cleared
    incident_status VARCHAR(20),               -- Current status of incident
    data_home CHAR(1),                         -- Code indicating data origin (local/state/federal system)
    orig_format CHAR(1),                       -- Tells us the original format of the data submission
    did BIGINT,                                -- Data ID for internal tracking or integration with external systems
    PRIMARY KEY (incident_id)                  -- Makes sure each incident record is uniquely identifiable
);

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE INCIDENT IS 'Table containing detailed information for each incident reported in NIBRS dataset';
COMMENT ON COLUMN INCIDENT.data_year IS 'Year in which the incident occurred';
COMMENT ON COLUMN INCIDENT.agency_id IS 'Identifier for the reporting law enforcement agency';
COMMENT ON COLUMN INCIDENT.incident_id IS 'Unique identifier for each incident';
COMMENT ON COLUMN INCIDENT.nibrs_month_id IS 'Identifier linking to the NIBRS reporting month';
COMMENT ON COLUMN INCIDENT.cargo_theft_flag IS 'Indicates whether the incident involved cargo theft';
COMMENT ON COLUMN INCIDENT.submission_date IS 'Date and time the incident was submitted to the system';
COMMENT ON COLUMN INCIDENT.incident_date IS 'Actual date of the incident';
COMMENT ON COLUMN INCIDENT.report_date_flag IS 'Flag indicating if the reported date differs from the incident date';
COMMENT ON COLUMN INCIDENT.incident_hour IS 'Hour of day when the incident occurred';
COMMENT ON COLUMN INCIDENT.cleared_except_id IS 'ID for reason of exceptional clearance';
COMMENT ON COLUMN INCIDENT.cleared_except_date IS 'Date when the incident was exceptionally cleared';
COMMENT ON COLUMN INCIDENT.incident_status IS 'Current status of the incident';
COMMENT ON COLUMN INCIDENT.data_home IS 'Code indicating origin of the data (local/state/federal)';
COMMENT ON COLUMN INCIDENT.orig_format IS 'Original format of the submitted data';
COMMENT ON COLUMN INCIDENT.did IS 'Data ID for internal tracking or integration with external systems';

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the OFFENSE_TYPE table if it already exists to avoid duplication
DROP TABLE IF EXISTS OFFENSE_TYPE CASCADE;

-- Create the OFFENSE_TYPE table
CREATE TABLE OFFENSE_TYPE (
    offense_code VARCHAR(10) NOT NULL,       -- Unique code representing a specific offense (Primary Key)
    offense_name VARCHAR(100),               -- Descriptive name of the offense
    crime_against VARCHAR(50),               -- Type of victim or target
    ct_flag CHAR(1),                         -- Tells us if the offense is related to cargo theft
    hc_flag CHAR(1),                         -- Tells us if the offense is classified as a hate crime
    hc_code VARCHAR(5),                      -- Code representing the specific hate crime bias
    offense_category_name VARCHAR(50),       -- General offense category
    offense_group CHAR(1),                   -- Group classification for reporting purposes
    PRIMARY KEY (offense_code)               -- Makes sure each offense code is unique
);


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE OFFENSE_TYPE IS 'Table containing offense types and associated classifications in the NIBRS dataset';
COMMENT ON COLUMN OFFENSE_TYPE.offense_code IS 'Unique identifier code for each offense type';
COMMENT ON COLUMN OFFENSE_TYPE.offense_name IS 'Descriptive name of the offense';
COMMENT ON COLUMN OFFENSE_TYPE.crime_against IS 'Type of victim or target for the offense';
COMMENT ON COLUMN OFFENSE_TYPE.ct_flag IS 'Flag indicating whether the offense involves cargo theft';
COMMENT ON COLUMN OFFENSE_TYPE.hc_flag IS 'Flag indicating if the offense is a hate crime';
COMMENT ON COLUMN OFFENSE_TYPE.hc_code IS 'Code representing the specific bias in hate crime offenses';
COMMENT ON COLUMN OFFENSE_TYPE.offense_category_name IS 'General category of the offense';
COMMENT ON COLUMN OFFENSE_TYPE.offense_group IS 'Group classification used for reporting purposes';

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the OFFENSE table if it already exists to avoid duplication
DROP TABLE IF EXISTS OFFENSE CASCADE;

-- Create the OFFENSE table
CREATE TABLE OFFENSE (
    data_year INT,                             -- Year the offense was recorded (useful for trend analysis)
    offense_id BIGINT NOT NULL,                -- Unique identifier for each offense (Primary Key)
    incident_id BIGINT,                        -- Identifier linking the offense to a specific INCIDENT
    offense_code VARCHAR(10),                  -- Code representing the type of offense (links to OFFENSE_TYPE)
    attempt_complete_flag CHAR(1),             -- Tells us whether the offense was attempted or completed
    location_id INT,                           -- Identifier for the location where the offense occurred
    num_premises_entered INT,                  -- Number of premises entered if the offense involved burglary or similar crimes
    method_entry_code VARCHAR(10),              -- Code indicating the method of entry
    PRIMARY KEY (offense_id),                   -- Ensures each offense record is uniquely identifiable
    FOREIGN KEY (incident_id) REFERENCES INCIDENT(incident_id), -- Links to INCIDENT table
    FOREIGN KEY (offense_code) REFERENCES OFFENSE_TYPE(offense_code) -- Links to OFFENSE_TYPE table for offense codes
);


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE OFFENSE IS 'Table containing offense records for each incident in the NIBRS dataset';
COMMENT ON COLUMN OFFENSE.data_year IS 'Year in which the offense was recorded';
COMMENT ON COLUMN OFFENSE.offense_id IS 'Unique identifier for each offense record';
COMMENT ON COLUMN OFFENSE.incident_id IS 'ID linking the offense to a specific incident';
COMMENT ON COLUMN OFFENSE.offense_code IS 'Code representing the type of offense, referencing OFFENSE_TYPE';
COMMENT ON COLUMN OFFENSE.attempt_complete_flag IS 'Indicates whether the offense was attempted or completed';
COMMENT ON COLUMN OFFENSE.location_id IS 'Identifier for the location where the offense occurred';
COMMENT ON COLUMN OFFENSE.num_premises_entered IS 'Number of premises entered for offenses like burglary';
COMMENT ON COLUMN OFFENSE.method_entry_code IS 'Code indicating the method of entry used in the offense';


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the VICTIM_TYPE table if it already exists to avoid conflicts
DROP TABLE IF EXISTS VICTIM_TYPE CASCADE;

-- Create the VICTIM_TYPE table
CREATE TABLE VICTIM_TYPE (
    victim_type_id INT NOT NULL,           -- Unique identifier for each victim type (Primary Key)
    victim_type_code CHAR(1),              -- Short code representing the victim type
    victim_type_name VARCHAR(50),          -- Descriptive name of the victim type
    PRIMARY KEY (victim_type_id)           -- Ensures each victim_type_id is unique
);


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE VICTIM_TYPE IS 'Table containing different victim types used in NIBRS dataset';
COMMENT ON COLUMN VICTIM_TYPE.victim_type_id IS 'Unique identifier for each victim type';
COMMENT ON COLUMN VICTIM_TYPE.victim_type_code IS 'Short code representing the victim type';
COMMENT ON COLUMN VICTIM_TYPE.victim_type_name IS 'Descriptive name of the victim type';

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the VICTIM table if it already exists to avoid duplication
DROP TABLE IF EXISTS VICTIM CASCADE;

-- Create the VICTIM table
CREATE TABLE VICTIM (
    data_year INT,                             -- Year the victim data was recorded
    victim_id BIGINT NOT NULL,                  -- Unique identifier for each victim (Primary Key)
    incident_id BIGINT,                        -- Identifier linking the victim to a specific INCIDENT
    victim_seq_num INT,                        -- Sequence number in case multiple victims are linked to the same incident
    victim_type_id INT,                         -- Foreign key referencing VICTIM_TYPE
    assignment_type_id INT,                    -- Identifier for the type of assignment or role in the incident
    activity_type_id INT,                      -- Identifier for victim activity at the time of the incident
    outside_agency_id INT,                      -- ID of an outside agency involved with the victim
    age_id INT,                                -- Identifier for age group or category
    age_num VARCHAR(10),                        -- Numeric age of the victim
    sex_code CHAR(1),                          -- Gender of the victim
    race_id INT,                               -- Identifier referencing a race lookup table
    ethnicity_id INT,                          -- Identifier referencing ETHNICITY table
    resident_status_code CHAR(1),                -- Indicates whether the victim is a resident or non-resident
    age_range_low_num INT,                     -- Lower bound of age range
    age_code_range_high INT,                   -- Upper bound of age range
    PRIMARY KEY (victim_id),                   -- Ensures each victim_id is unique
    FOREIGN KEY (incident_id) REFERENCES INCIDENT(incident_id),       -- Links victim to a specific incident
    FOREIGN KEY (victim_type_id) REFERENCES VICTIM_TYPE(victim_type_id) -- Links victim to victim type
);


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE VICTIM IS 'Table containing victim details for each incident in the NIBRS dataset';
COMMENT ON COLUMN VICTIM.data_year IS 'Year in which the victim information was recorded';
COMMENT ON COLUMN VICTIM.victim_id IS 'Unique identifier for each victim';
COMMENT ON COLUMN VICTIM.incident_id IS 'ID linking the victim to a specific incident';
COMMENT ON COLUMN VICTIM.victim_seq_num IS 'Sequence number for multiple victims in the same incident';
COMMENT ON COLUMN VICTIM.victim_type_id IS 'Identifier for the type of victim, referencing VICTIM_TYPE';
COMMENT ON COLUMN VICTIM.assignment_type_id IS 'Identifier for the type of assignment or role in the incident';
COMMENT ON COLUMN VICTIM.activity_type_id IS 'Identifier for victim activity at the time of the incident';
COMMENT ON COLUMN VICTIM.outside_agency_id IS 'ID of an outside agency involved with the victim';
COMMENT ON COLUMN VICTIM.age_id IS 'Identifier for the age group or category of the victim';
COMMENT ON COLUMN VICTIM.age_num IS 'Numeric age of the victim';
COMMENT ON COLUMN VICTIM.sex_code IS 'Gender of the victim';
COMMENT ON COLUMN VICTIM.race_id IS 'Identifier referencing a race lookup table';
COMMENT ON COLUMN VICTIM.ethnicity_id IS 'Identifier referencing ETHNICITY table';
COMMENT ON COLUMN VICTIM.resident_status_code IS 'Indicates if the victim is a resident or non-resident';
COMMENT ON COLUMN VICTIM.age_range_low_num IS 'Lower bound of victim age range';
COMMENT ON COLUMN VICTIM.age_code_range_high IS 'Upper bound of victim age range';

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the VICTIM_OFFENSE table if it already exists to avoid duplication
DROP TABLE IF EXISTS VICTIM_OFFENSE CASCADE;

-- Create the VICTIM_OFFENSE table
CREATE TABLE VICTIM_OFFENSE (
    data_year INT,                             -- Year the victim-offense record was recorded
    victim_id BIGINT,                          -- Foreign key referencing a specific victim
    offense_id BIGINT,                         -- Foreign key referencing a specific offense
    PRIMARY KEY (victim_id, offense_id),       -- Composite primary key ensures uniqueness of each victim-offense pair
    FOREIGN KEY (victim_id) REFERENCES VICTIM(victim_id),   -- Links to VICTIM table
    FOREIGN KEY (offense_id) REFERENCES OFFENSE(offense_id) -- Links to OFFENSE table
);


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE VICTIM_OFFENSE IS 'Table linking victims to offenses, indicating which victims were involved in which offenses';
COMMENT ON COLUMN VICTIM_OFFENSE.data_year IS 'Year in which the victim-offense record was recorded';
COMMENT ON COLUMN VICTIM_OFFENSE.victim_id IS 'Identifier for the victim, referencing VICTIM table';
COMMENT ON COLUMN VICTIM_OFFENSE.offense_id IS 'Identifier for the offense, referencing OFFENSE table';

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the PROPERTY_LOSS_TYPE table if it already exists to avoid duplication
DROP TABLE IF EXISTS PROPERTY_LOSS_TYPE CASCADE;

-- Create the PROPERTY_LOSS_TYPE table
CREATE TABLE PROPERTY_LOSS_TYPE (
    prop_loss_id INT NOT NULL,             -- Unique identifier for each property loss type (Primary Key)
    prop_loss_name VARCHAR(50),            -- Short descriptive name of the property loss
    prop_loss_desc VARCHAR(100),           -- Detailed description of the property loss type
    PRIMARY KEY (prop_loss_id)             -- Ensures each prop_loss_id is unique
);


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE PROPERTY_LOSS_TYPE IS 'Table containing types of property loss in NIBRS incidents';
COMMENT ON COLUMN PROPERTY_LOSS_TYPE.prop_loss_id IS 'Unique identifier for each property loss type';
COMMENT ON COLUMN PROPERTY_LOSS_TYPE.prop_loss_name IS 'Short descriptive name of the property loss';
COMMENT ON COLUMN PROPERTY_LOSS_TYPE.prop_loss_desc IS 'Detailed description of the property loss type';

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the PROPERTY_DESC_TYPE table if it already exists to avoid duplication
DROP TABLE IF EXISTS PROPERTY_DESC_TYPE CASCADE;

-- Create the PROPERTY_DESC_TYPE table
CREATE TABLE PROPERTY_DESC_TYPE (
    prop_desc_id INT NOT NULL,            -- Unique identifier for each property description type (Primary Key)
    prop_desc_name VARCHAR(50),           -- Descriptive name of the property type
    prop_desc_code CHAR(10),              -- Short code representing the property description
    PRIMARY KEY (prop_desc_id)            -- Ensures each prop_desc_id is unique
);


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE PROPERTY_DESC_TYPE IS 'Table containing types of property descriptions in NIBRS incidents';
COMMENT ON COLUMN PROPERTY_DESC_TYPE.prop_desc_id IS 'Unique identifier for each property description type';
COMMENT ON COLUMN PROPERTY_DESC_TYPE.prop_desc_name IS 'Descriptive name of the property type';
COMMENT ON COLUMN PROPERTY_DESC_TYPE.prop_desc_code IS 'Short code representing the property description';


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the PROPERTY table if it already exists to avoid duplication
DROP TABLE IF EXISTS PROPERTY CASCADE;

-- Create the PROPERTY table
CREATE TABLE PROPERTY (
    data_year INT,                              -- Year the property data was recorded
    property_id BIGINT NOT NULL,                 -- Unique identifier for each property record (Primary Key)
    incident_id BIGINT,                         -- Identifier linking the property to a specific INCIDENT
    prop_loss_id INT,                           -- Foreign key referencing PROPERTY_LOSS_TYPE
    prop_desc_id INT,                           -- Foreign key referencing PROPERTY_DESC_TYPE
    PRIMARY KEY (property_id),                  -- Makes sure each property record is uniquely identifiable
    FOREIGN KEY (incident_id) REFERENCES INCIDENT(incident_id),                -- Links property to a specific incident
    FOREIGN KEY (prop_loss_id) REFERENCES PROPERTY_LOSS_TYPE(prop_loss_id),     -- Links to property loss type
    FOREIGN KEY (prop_desc_id) REFERENCES PROPERTY_DESC_TYPE(prop_desc_id)      -- Links to property description
);


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE PROPERTY IS 'Table containing property details associated with NIBRS incidents';
COMMENT ON COLUMN PROPERTY.data_year IS 'Year in which the property data was recorded';
COMMENT ON COLUMN PROPERTY.property_id IS 'Unique identifier for each property record';
COMMENT ON COLUMN PROPERTY.incident_id IS 'Identifier linking the property to a specific incident';
COMMENT ON COLUMN PROPERTY.prop_loss_id IS 'Identifier for type of property loss, referencing PROPERTY_LOSS_TYPE';
COMMENT ON COLUMN PROPERTY.prop_desc_id IS 'Identifier for type of property description, referencing PROPERTY_DESC_TYPE';

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Drop the PROPERTY_DESC table if it already exists to avoid duplication
DROP TABLE IF EXISTS PROPERTY_DESC CASCADE;

-- Create the PROPERTY_DESC table
CREATE TABLE PROPERTY_DESC (
    data_year INT,                             -- Year the property description record was recorded
    property_id BIGINT,                        -- Foreign key linking to a specific property record
    prop_desc_id INT,                          -- Foreign key linking to PROPERTY_DESC_TYPE for the property description
    property_value VARCHAR(50),                -- Value of the property item
    date_recovered DATE,                       -- Date the property was recovered
    PRIMARY KEY (property_id, prop_desc_id),   -- Primary key ensures uniqueness for each property-description combination
    FOREIGN KEY (property_id) REFERENCES PROPERTY(property_id),          -- Links to the PROPERTY table
    FOREIGN KEY (prop_desc_id) REFERENCES PROPERTY_DESC_TYPE(prop_desc_id) -- Links to PROPERTY_DESC_TYPE for standardized property descriptions
);


Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

In [ ]:
%%sql
-- Comments for table and columns
COMMENT ON TABLE PROPERTY_DESC IS 'Table linking property records to their specific descriptions and values';
COMMENT ON COLUMN PROPERTY_DESC.data_year IS 'Year in which the property description record was recorded';
COMMENT ON COLUMN PROPERTY_DESC.property_id IS 'Identifier linking to a specific property record';
COMMENT ON COLUMN PROPERTY_DESC.prop_desc_id IS 'Identifier for property description type, referencing PROPERTY_DESC_TYPE';
COMMENT ON COLUMN PROPERTY_DESC.property_value IS 'Value or description of the property item';
COMMENT ON COLUMN PROPERTY_DESC.date_recovered IS 'Date on which the property was recovered';

Running query in 'postgresql://student@/nibrs_db'

++
||
++
++

### 3. Bulk Data Loading
In this part of the notebook, we are importing the raw NIBRS CSV files into the database tables we created earlier.

Here’s what’s happening overall:

The COPY commands load each CSV file into its matching table — for example:

NIBRS_OFFENSE.csv → OFFENSE table

NIBRS_VICTIM.csv → VICTIM table

NIBRS_OFFENDER.csv → OFFENDER table

This process transfers the cleaned and structured data from the CSV files into the PostgreSQL database so it can be queried efficient

In [ ]:
%%sql
COPY ARREST_TYPE
-- Loading data from NIBRS corresponding csv file into the ARREST_TYPE table that was created
-- CSV HEADER accounts for the header row in the data
FROM '/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ARREST_TYPE.csv'
CSV HEADER;

Running query in 'postgresql://student@/nibrs_db'

3 rows affected.

++
||
++
++

In [ ]:
%%sql
COPY ARRESTEE
-- Loading data from NIBRS_incident.csv into the ARRESTEE table that was created
-- CSV HEADER accounts for the header row in the data
FROM '/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ARRESTEE.csv'
CSV HEADER;


Running query in 'postgresql://student@/nibrs_db'

56111 rows affected.

++
||
++
++

In [ ]:
%%sql
COPY BIAS_LIST
-- Loading data from NIBRS corresponding csv file into the BIAS_LIST table that was created
-- CSV HEADER accounts for the header row in the data
FROM '/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_BIAS_LIST.csv'
CSV HEADER;


Running query in 'postgresql://student@/nibrs_db'

36 rows affected.

++
||
++
++

In [ ]:
%%sql
COPY BIAS_MOTIVATION
-- Loading data from NIBRS corresponding csv file into the BIAS_MOTIVATION table that was created
-- CSV HEADER accounts for the header row in the data
FROM '/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_BIAS_MOTIVATION.csv'
CSV HEADER;

Running query in 'postgresql://student@/nibrs_db'

301364 rows affected.

++
||
++
++

In [ ]:
%%sql
COPY ETHNICITY
-- Loading data from NIBRS corresponding csv file into the ETHNICITY table that was created
-- CSV HEADER accounts for the header row in the data
FROM '/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ETHNICITY.csv'
CSV HEADER;

Running query in 'postgresql://student@/nibrs_db'

5 rows affected.

++
||
++
++

In [ ]:
%%sql
-- Loading data from NIBRS corresponding csv file into the INCIDENT table that was created
-- CSV HEADER accounts for the header row in the data
COPY INCIDENT
FROM '/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_incident.csv'
CSV HEADER;

Running query in 'postgresql://student@/nibrs_db'

264904 rows affected.

++
||
++
++

In [ ]:
%%sql
COPY OFFENSE_TYPE
-- Loading data from NIBRS corresponding csv file into the OFFENSE_TYPE table that was created
-- CSV HEADER accounts for the header row in the data
FROM '/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_OFFENSE_TYPE.csv'
CSV HEADER;

Running query in 'postgresql://student@/nibrs_db'

86 rows affected.

++
||
++
++

In [ ]:
%%sql
COPY OFFENSE
-- Loading data from NIBRS corresponding csv file into the OFFENSE table that was created
-- CSV HEADER accounts for the header row in the data
FROM '/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_OFFENSE.csv'
CSV HEADER;

Running query in 'postgresql://student@/nibrs_db'

301348 rows affected.

++
||
++
++

In [ ]:
%%sql
COPY VICTIM_TYPE
-- Loading data from NIBRS corresponding csv file into the VICTIM_TYPE table that was created
-- CSV HEADER accounts for the header row in the data
FROM '/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_VICTIM_TYPE.csv'
CSV HEADER;

Running query in 'postgresql://student@/nibrs_db'

9 rows affected.

++
||
++
++

In [ ]:
%%sql
COPY VICTIM
-- Loading data from NIBRS corresponding csv file into the VICTIM table that was created
-- CSV HEADER accounts for the header row in the data
FROM '/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_VICTIM.csv'
CSV HEADER;

Running query in 'postgresql://student@/nibrs_db'

305550 rows affected.

++
||
++
++

In [ ]:
%%sql
COPY VICTIM_OFFENSE
-- Loading data from NIBRS corresponding csv file into the VICTIM_OFFENSE table that was created
-- CSV HEADER accounts for the header row in the data
FROM '/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_VICTIM_OFFENSE.csv'
CSV HEADER;

Running query in 'postgresql://student@/nibrs_db'

336674 rows affected.

++
||
++
++

### 4. Data validation

In [ ]:
%%sql
-- Count rows in ARRESTEE table
SELECT COUNT(*) AS total_rows FROM ARRESTEE;

Running query in 'postgresql://student@/nibrs_db'

1 rows affected.

total_rows
56111


In [ ]:
%%sql
-- Count rows in ARREST_TYPE table
SELECT COUNT(*) AS total_rows FROM ARREST_TYPE;

Running query in 'postgresql://student@/nibrs_db'

1 rows affected.

total_rows
3


In [ ]:
%%sql
-- Count rows in BIAS_LIST table
SELECT COUNT(*) AS total_rows FROM BIAS_LIST;

Running query in 'postgresql://student@/nibrs_db'

1 rows affected.

total_rows
36


In [ ]:
%%sql
-- Count rows in BIAS_MOTIVATION table
SELECT COUNT(*) AS total_rows FROM BIAS_MOTIVATION;

Running query in 'postgresql://student@/nibrs_db'

1 rows affected.

total_rows
301364


In [ ]:
%%sql
-- Count rows in ETHNICITY table
SELECT COUNT(*) AS total_rows FROM ETHNICITY;

Running query in 'postgresql://student@/nibrs_db'

1 rows affected.

total_rows
5


In [ ]:
%%sql
-- Count rows in each INCIDENT table
SELECT COUNT(*) AS total_rows_INCIDENT FROM INCIDENT;


Running query in 'postgresql://student@/nibrs_db'

1 rows affected.

total_rows_incident
264904


In [ ]:
%%sql
-- Count rows in OFFENSE table
SELECT COUNT(*) AS total_rows FROM OFFENSE;


Running query in 'postgresql://student@/nibrs_db'

1 rows affected.

total_rows
301348


In [ ]:
%%sql

-- Count rows in OFFENSE_TYPE table
SELECT COUNT(*) AS total_rows FROM OFFENSE_TYPE;


Running query in 'postgresql://student@/nibrs_db'

1 rows affected.

total_rows
86


In [ ]:
%%sql

-- Count rows in VICTIM table
SELECT COUNT(*) AS total_rows FROM VICTIM;

Running query in 'postgresql://student@/nibrs_db'

1 rows affected.

total_rows
305550


In [ ]:
%%sql
-- Count rows in VICTIM_OFFENSE table
SELECT COUNT(*) AS total_rows FROM VICTIM_OFFENSE;

Running query in 'postgresql://student@/nibrs_db'

1 rows affected.

total_rows
336674


In [ ]:
%%sql
-- Count rows in VICTIM_TYPE table
SELECT COUNT(*) AS total_rows FROM VICTIM_TYPE;


Running query in 'postgresql://student@/nibrs_db'

1 rows affected.

total_rows
9


Compare with original CSV row counts

In [ ]:
# data folder
!cd /home/ubuntu/Assignment_2/NIBRS_files/datafiles

In [ ]:
# Count rows in all CSV files
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ARRESTEE.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ARREST_TYPE.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_BIAS_LIST.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_BIAS_MOTIVATION.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ETHNICITY.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_incident.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_OFFENSE.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_OFFENSE_TYPE.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_VICTIM.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_VICTIM_OFFENSE.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_VICTIM_TYPE.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROPERTY.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROPERTY_DESC.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROP_DESC_TYPE.csv"
!wc -l "/home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROP_LOSS_TYPE.csv"

56112 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ARRESTEE.csv
4 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ARREST_TYPE.csv
37 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_BIAS_LIST.csv
301365 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_BIAS_MOTIVATION.csv
6 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_ETHNICITY.csv
264905 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_incident.csv
301349 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_OFFENSE.csv
87 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_OFFENSE_TYPE.csv
305551 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_VICTIM.csv
336675 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_VICTIM_OFFENSE.csv
10 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/NIBRS_VICTIM_TYPE.csv
242273 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROPERTY.csv
292456 /home/ubuntu/Assignment_2/NIBRS_files/datafiles/Sponsor Request/NIBRS_PROPERTY_DESC.csv
69

In [ ]:
%%bash
# Showing a comparison table of SQL vs CSV row counts for the main NIBRS tables
# This helps us confirm that all data was loaded correctly

echo "Comparison of SQL vs CSV Row Counts for Main NIBRS Tables"
echo
echo -e "Table\tSQL Rows\tCSV Rows (minus header)\tDifference (SQL - CSV)"
echo -e "ARRESTEE\t56111\t56111\t0"
echo -e "ARREST_TYPE\t3\t3\t0"
echo -e "BIAS_LIST\t36\t36\t0"
echo -e "BIAS_MOTIVATION\t301364\t301364\t0"
echo -e "ETHNICITY\t5\t5\t0"
echo -e "INCIDENT\t264904\t264904\t0"
echo -e "OFFENSE\t301348\t301348\t0"
echo -e "OFFENSE_TYPE\t86\t86\t0"
echo -e "VICTIM\t305550\t305550\t0"
echo -e "VICTIM_OFFENSE\t336674\t336674\t0"
echo -e "VICTIM_TYPE\t9\t9\t0"


Comparison of SQL vs CSV Row Counts for Main NIBRS Tables

Table	SQL Rows	CSV Rows (minus header)	Difference (SQL - CSV)
ARRESTEE	56111	56111	0
ARREST_TYPE	3	3	0
BIAS_LIST	36	36	0
BIAS_MOTIVATION	301364	301364	0
ETHNICITY	5	5	0
INCIDENT	264904	264904	0
OFFENSE	301348	301348	0
OFFENSE_TYPE	86	86	0
VICTIM	305550	305550	0
VICTIM_OFFENSE	336674	336674	0
VICTIM_TYPE	9	9	0


### 5. Sponsor request
The four NIBRS property-related files—NIBRS_PROPERTY.csv, NIBRS_PROP_LOSS_TYPE.csv, NIBRS_PROP_DESC_TYPE.csv, and NIBRS_PROPERTY_DESC.csv—can be effectively integrated into the database by establishing a clear structure and relationships. The NIBRS_PROPERTY.csv file should serve as the main PROPERTY table, linked to the existing INCIDENT table via INCIDENT_ID and including a reference to property loss type. The NIBRS_PROP_LOSS_TYPE.csv and NIBRS_PROP_DESC_TYPE.csv files should be used to create lookup tables for loss types and property description types, respectively, standardizing categorical fields and ensuring consistency. Finally, the NIBRS_PROPERTY_DESC.csv file should form a PROPERTY_DESC table that links each property to its detailed descriptions, allowing for multiple descriptions per property. By creating the lookup tables first, followed by the main property and description tables, and establishing proper foreign key relationships, this integration will maintain data integrity, support 1-to-many relationships, and facilitate more accurate analysis of property-related incidents.

### 6. Basic data analysis
Query 1: Top and Bottom Offenses by Frequency

In [ ]:
%%sql

-- Top 5 offenses
SELECT o.offense_code, ot.offense_name, ot.offense_category_name, COUNT(*) AS total_offenses
FROM OFFENSE o
JOIN OFFENSE_TYPE ot ON o.offense_code = ot.offense_code
GROUP BY o.offense_code, ot.offense_name, ot.offense_category_name
ORDER BY total_offenses DESC
LIMIT 5;


Running query in 'postgresql://student@/nibrs_db'

5 rows affected.

offense_code,offense_name,offense_category_name,total_offenses
13B,Simple Assault,Assault Offenses,47719
290,Destruction/Damage/Vandalism of Property,Destruction/Damage/Vandalism of Property,42638
23C,Shoplifting,Larceny/Theft Offenses,31801
240,Motor Vehicle Theft,Motor Vehicle Theft,25744
23H,All Other Larceny,Larceny/Theft Offenses,20869


The results of the query summarize the total counts of each specific offense type in the dataset. The most common is 13B, Simple Assault, which has 47,719 occurrences, while 290 – Destruction/Damage/Vandalism of Property comes second with 42,638 cases. This means that both violent and property offenses are quite high in this dataset. Of the theft-related crimes, 23C – Shoplifting contributes 31,801 offenses and 23H – All Other Larceny contributes 20,869, while 240 – Motor Vehicle Theft appears 25,744 times. It would seem that these results emphasize the prevalence of simple assaults and property crimes within this dataset, meaning that law enforcement and crime prevention efforts may need to take such common offense types into consideration, as they would make up the largest volume of criminal activity.

In [ ]:
%%sql

-- Bottom 5 offenses
SELECT o.offense_code, ot.offense_name, ot.offense_category_name, COUNT(*) AS total_offenses
FROM OFFENSE o
JOIN OFFENSE_TYPE ot ON o.offense_code = ot.offense_code
GROUP BY o.offense_code, ot.offense_name, ot.offense_category_name
ORDER BY total_offenses ASC
LIMIT 5;


Running query in 'postgresql://student@/nibrs_db'

5 rows affected.

offense_code,offense_name,offense_category_name,total_offenses
39C,Gambling Equipment Violation,Gambling Offenses,1
64B,"Human Trafficking, Involuntary Servitude",Human Trafficking,2
510,Bribery,Bribery,3
39A,Betting/Wagering,Gambling Offenses,5
09B,Negligent Manslaughter,Homicide Offenses,17


Results of the query provide the breakdown of offenses by code, name, category, and total occurrences. Within the offenses listed, 09B – Negligent Manslaughter occurred the most with a count of 17, which would also suggest that the most common offense in this subset of data is related to homicide. Gambling-related offenses come next, with 39A – Betting/Wagering occurring 5 times and 39C – Gambling Equipment Violation occurring once, suggesting that betting activities are more prevalent than equipment violations. Offenses such as 510 – Bribery, which had 3 occurrences, and 64B – Human Trafficking, Involuntary Servitude with 2 occurrences, are comparatively rare. Overall, results indicate that serious violent crimes dominate the dataset, while specialized offenses like human trafficking and bribery make up fewer of the occurrences, which can reflect either reporting or the prevalence of the crimes.

Query 2: Victim Distribution by Sex, Race, and Location

In [ ]:
%%sql

SELECT v.sex_code, v.race_id, o.location_id, COUNT(*) AS num_victims
FROM VICTIM v
JOIN VICTIM_TYPE vt ON v.victim_type_id = vt.victim_type_id
JOIN VICTIM_OFFENSE vo ON v.victim_id = vo.victim_id
JOIN OFFENSE o ON vo.offense_id = o.offense_id
WHERE vt.victim_type_name = 'Individual'
GROUP BY v.sex_code, v.race_id, o.location_id
ORDER BY num_victims DESC;


Running query in 'postgresql://student@/nibrs_db'

509 rows affected.

sex_code,race_id,location_id,num_victims
F,20,35,28091
F,10,35,23975
M,10,35,20814
M,20,35,17087
M,20,25,11400
F,20,25,11019
M,10,25,10935
F,20,33,10794
M,20,33,10064
M,10,33,8988


The query results summarize the number of victims by sex, race, and location. It can be observed from the data that female victims of race 20 at location 35 are the most frequent, with 28,091 reported victims, followed by female victims of race 10 at the same location with 23,975 victims. For male victims, the counts are generally a little lower, the highest count being 20,814 male victims of race 10 at location 35. As we go down to other locations, the counts decrease, therefore showing that location 35 has the highest number of reported victims across several sex and race categories. These findings point out both demographic and geographic patterns in victimization and point out that sex, race, and location are important factors for understanding the distribution of victims within this dataset.

Query 3: Victims with Multiple Offenses in a Single Incident

In [ ]:
%%sql

SELECT
    v.victim_id,
    v.sex_code,
    v.race_id,
    v.ethnicity_id,
    o.incident_id,
    COUNT(*) AS num_offenses
FROM VICTIM v
JOIN VICTIM_OFFENSE vo ON v.victim_id = vo.victim_id
JOIN OFFENSE o ON vo.offense_id = o.offense_id
GROUP BY
    v.victim_id,
    v.sex_code,
    v.race_id,
    v.ethnicity_id,
    o.incident_id
HAVING COUNT(*) >= 4
ORDER BY num_offenses DESC;


Running query in 'postgresql://student@/nibrs_db'

48 rows affected.

victim_id,sex_code,race_id,ethnicity_id,incident_id,num_offenses
215772526,F,10,20,193886004,5
218870139,F,10,50,196571277,5
215045083,X,99,50,193251445,5
224991116,M,10,20,201899218,4
217872524,M,20,50,195710005,4
219974277,M,10,20,197534476,4
219975235,F,20,50,197535240,4
230463251,F,20,40,206680680,4
215045086,U,98,40,193251445,4
220619529,X,99,50,198097989,4


The results of the query summarize incidents by victim demographics and the number of offenses experienced per incident. It can be observed that in a single incident, some victims experience a really high number of offenses; for instance, a single victim may experience a maximum of 5 offenses, while others may experience 4. Victims span many sex codes, including male, female, unknown, and unspecified, as well as many races and ethnicities, indicating that multi-offense incidents can affect a wide range of populations. Most victims experienced 4 offenses, indicating that though truly extreme cases are uncommon, multi-offense incidents are common enough to deserve consideration. This underlines the importance of consideration of both incident-level severity and demographic contexts in the analysis of victim involvement in crimes.

Query 4: Incidents with More Than 4 Victims

In [ ]:
%%sql
WITH victim_count AS (
  SELECT incident_id
  FROM victim
  GROUP BY incident_id
  HAVING COUNT(victim_id) >= 4
)

SELECT
    victim.incident_id AS incident,
    victim.sex_code AS sex,
    victim.race_id AS race,
    victim.ethnicity_id AS ethnicity,
    COUNT(victim.victim_id) AS victims_involved

FROM victim
JOIN victim_count
    ON victim.incident_id = victim_count.incident_id

GROUP BY victim.incident_id, victim.sex_code, victim.race_id, victim.ethnicity_id
ORDER BY victims_involved DESC;


Running query in 'postgresql://student@/nibrs_db'

6161 rows affected.

incident,sex,race,ethnicity,victims_involved
204099204,X,99,50,69
206454316,U,98,40,33
207013637,U,98,40,22
205710499,F,98,40,12
206059797,F,20,50,12
201826309,X,99,50,11
193251054,F,20,50,11
198784314,M,10,50,10
203057062,M,20,20,10
197597837,M,20,50,10


The results of the query summarize incidents by victim demographics and the number of victims involved. The data reveals that some incidents involve an extremely high number of victims: the highest incidence reports 69 victims from a single incident, incident number 204099204, while others report as many as 33 and 22 victims. Victims fall under various sex codes, from male to female, unknown to other or unspecified codes; they encompass people of various races and ethnicities, indicating a wide population within which such incidents may occur. Most incidents reported 10 to 12 victims, meaning that though large-scale incidents may prevail, the majority deal with smaller groups. This, in turn, indicates the difference in incident severity and the relevance of demographic contexts in analyzing the involvement of victims in crimes.